In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\harsh\\OneDrive\\Desktop\\Job\\ResumeProjects\\SterileBrowsing\\SterileBrowsingMedicalImgs'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnImageClassification.constants import *
from cnnImageClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(Path(self.config.data_ingestion.unzip_dir), "GestureDataset","data")
        print(training_data)
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [19]:
import time


In [20]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [22]:
import cv2
import numpy as np
import glob
from sklearn.model_selection import train_test_split

In [29]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )
    
    def prepare_training_testing_data(self):

        # assume this is the path to the dataset

        dataset_root = self.config.training_data

        max_images_per_gesture = 1600
        loaded_images = []
        output_vectors = []

        # List of gesture folders (0 to 5)
        
        list_of_gestures = ['blank', 'ok', 'thumbsup', 'thumbsdown', 'fist', 'five']

        # Load images and create one-hot encoded labels
        for gesture in list_of_gestures:
            # Construct the path to the current gesture folder
            gesture_path = os.path.join(dataset_root, gesture, '*')
            gest_image_paths = glob.glob(gesture_path)
            
            # Load images and resize them
            for i, image_path in enumerate(gest_image_paths):
                if i < max_images_per_gesture:
                    image = cv2.imread(image_path)
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    gray_image = cv2.resize(gray_image, (100, 120))
                    loaded_images.append(gray_image)
                    
                    # Create the one-hot encoded output vector
                    one_hot_vector = [0] * len(list_of_gestures)  # Initialize vector with zeros
                    gesture_index = list_of_gestures.index(gesture)
                    one_hot_vector[gesture_index] = 1  # Set the index corresponding to the current gesture to 1
                    output_vectors.append(one_hot_vector)
        self.X = np.array(loaded_images)
        self.y = np.array(output_vectors)
        print(self.X.shape)
        # Split data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=4
        )

        # Reshape the data
        self.X_train = self.X_train.reshape(
            self.X_train.shape[0],
            self.config.params_image_size[0],
            self.config.params_image_size[1],
            self.config.params_image_size[2]
        )
        self.X_test = self.X_test.reshape(
            self.X_test.shape[0],
            self.config.params_image_size[0],
            self.config.params_image_size[1],
            self.config.params_image_size[2]
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
       
        self.model.fit(
            self.X_train, self.y_train,
            batch_size=self.config.params_batch_size,
            epochs=self.config.params_epochs,
            verbose=1,
            validation_data=(self.X_test, self.y_test),
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [30]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.prepare_training_testing_data()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-10-28 17:49:47,562: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Reading yaml file from config\config.yaml loaded suucessfully]
[2024-10-28 17:49:47,564: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Reading yaml file from params.yaml loaded suucessfully]
[2024-10-28 17:49:47,565: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Directory created at artifacts]
[2024-10-28 17:49:47,565: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Directory created at artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-28 17:49:47,566: INFO: common{c:\users\harsh\onedrive\desktop\job\r